## Лабораторная работа №1 | Вариант 4

Выполнил: Кудрявов Леонид Вадимович М8О-08Б-22

### 1. Numpy

1. Найти индексы не нулевых элементов массива [1,2,0,0,4,0]

In [ ]:
arr = [1, 2, 0, 0, 4, 0]
np.nonzero(arr)

2. Создайте единичную матрицу размером 3х3

In [ ]:
np.identity(3)

3. Создайте матрицу-шахматную доску 8х8 используя функцию `tile`

In [ ]:
base_pattern = np.array([[0, 1],
                         [1, 0]])
np.tile(base_pattern, (4, 4))

4. Дан 1D массив, удалите все элементы которые входят в диапазон `[3, 8]` inplace

In [ ]:
arr = [0, 4, 4, 2, 1, 4, 8, 7, 10, 15]
arr = np.array(arr)
mask = (arr > 8) | (arr < 3)
arr = arr[mask]
arr

5. Создайте случайный вектор размером 10 элементов и замените максимальный элемент на 0

In [ ]:
vec = np.random.rand(10)
vec[vec.argmax()] = 0
vec

6. Как распечатать все значения в массиве?

In [ ]:
print(*vec)

7. Рассмотрим случайный вектор с формой 100х2, представляющей координаты, и найдем расстояния от точки к точке

In [ ]:
vec = np.random.randint(-100, 100, (100, 2))
X = vec[:, np.newaxis, :]
Y = vec[np.newaxis, :, :]
diff = X - Y
np.sqrt(np.sum(diff**2, axis=-1))

8. Как конвертировать массив float(32 bit) к массиву целых чисел (integer 32 bit) in place?

In [ ]:
arr = np.array(np.random.uniform(0, 100, 50), dtype=np.float32)
print(arr.dtype)
converted_arr = arr.astype(np.int32)
converted_arr.dtype

9. Как прочитать заданный файл?
```
1, 2, 3, 4, 5
6,  ,  , 7, 8
 ,  , 9,10,11
```

In [ ]:
import re


wtf = """1, 2, 3, 4, 5
6,  ,  , 7, 8
 ,  , 9,10,11"""
np.array([float(elt)
          if elt.strip()
          else np.nan
          for elt in re.split(r"[,\n]", wtf)])

10. Какой эквивалент enumerate для numpy массивов?

In [ ]:
list(np.ndenumerate(np.random.randint(low=1, high=6, size=8)))

### 2. Pandas

Найдите ответы на вопросы или выполните действия с предложенным датасетом 

Файл: `russian-housing-market.csv`

```
price_doc - Цена продажи
id - Id транзакции
timestamp - Дата транзакции
full_sq - суммарная площадь в квадратных метрах, включая лоджию, валконы и отдельные нежилые помещения
life_sq - жилая площаль в квадратных метрах, исключая лоджии, балконы и прочие нежилые помещения
floor - для апартаментов, этаж помещения
max_floor - количество этажей в здании
material - материал стен
build_year - год постройки
num_room - количество жилых комнат
kitch_sq - площадь кухни
state - состояние апартаментов
product_type - покупка или инвестиция собственника-арендатора
sub_area - название района
```

In [13]:
import pandas as pd

In [ ]:
df_all = pd.read_csv("russian-housing-market.csv")
df_all.head()

1. Посчитайте количество записей в датасете с отсутствующей информацией о жилой площади жилого помещения

In [ ]:
df_all[df_all["life_sq"].isna()].shape[0]

2. Предложите способ заменить отсутствующие значения из пункта 1 и реализуйте его. Способ нужно аргументировать(сказать почему это достаточно хорошее приближение)

In [ ]:
import plotly.graph_objs as go

df_all["life_sq"] = df_all["life_sq"].median()

# Заполняем медианным значением, так как это позволяет лучше обработать
#  выбросы => приближение значений будет достаточно точным

3. Какие значения имеет 25, 50 и 75 перцентили полной площади помещения

In [ ]:
df_all["full_sq"].quantile([0.25, 0.5, 0.75])

4. Посчитайте среднюю стоимость продажи помещений с общей площадью от 50 до 70 квадратных метров

In [ ]:
df_all[(df_all["full_sq"] >= 50)
        & (df_all["full_sq"] <= 70)]["price_doc"].mean()

5. Какая максимальная этажность здания представлена в датасете

In [ ]:
df_all["max_floor"].max()

6. Выявите район с самым дорогим жильем

In [ ]:
df = (df_all
      .dropna(subset="price_doc")
      .groupby("sub_area")["price_doc"]
      .mean()
      .reset_index())
df.loc[df["price_doc"] == df["price_doc"].max(), "sub_area"]

7. Посчитайте среднюю стоимость за квадратный метр жилья в Митино(Mitino)

In [ ]:
df_all[df_all["sub_area"] == "Mitino"]["price_doc"].mean()

8. Посчитайте среднюю площадь комнаты в Бибирево

In [ ]:
df = df_all[df_all["sub_area"] == "Bibirevo"].dropna(subset=["life_sq", "num_room"])
(df["life_sq"] / df["num_room"]).mean()

9. Проанализируйте есть ли разница в цене между квартирами на 1 и 9 этажах

In [ ]:
import plotly.express as px


df = df_all[((df_all["floor"] == 1) | (df_all["floor"] == 9))
             & (df_all["price_doc"] <= 20 * 10**6)].dropna(subset="price_doc").sort_values(by="price_doc")
fig = go.Figure()
df_1 = df[df["floor"] == 1]
fig.add_trace(go.Histogram(x=df_1["price_doc"],
                           name="1st floor",
                           marker=dict(color=f"rgba(10, 186, 181, 0.8)")))

df_9 = df[df["floor"] == 9]
fig.add_trace(go.Histogram(x=df_9["price_doc"],
                           name="9th floor",
                           marker=dict(color=f"rgba(218, 112, 214, 0.5)")))
fig.update_layout(barmode="overlay",
                  bargap=0.2)

In [28]:
# Да, разница есть, первые этажи, в целом, гораздо дешевле 9-х

10. Найдите минимальную стоимость квартиры с жилой площадью не менее 60 квадратных метров, не менее чем 3мя комнатами и не ниже чем на 5 этаже

In [ ]:
df_all[(df_all["life_sq"] >= 60)
       & (df_all["num_room"] >= 3)
       & (df_all["floor"] >= 5)]["price_doc"].min()

### 3. Визуализация

Используя библиотеки matplotlib/sns/plotly/pandas и набор данных представленный ранее визуализируйте

1. Как менялась средняя цена квартиры в Нагатинском Затоне(Nagatinski Zaton) с течением времени

In [ ]:
df = df_all.copy()
df = df[df["sub_area"] == "Nagatinskij Zaton"]
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d")
df["period"] = df["timestamp"].dt.to_period("M")
df = df.groupby("period")["price_doc"].mean().reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df["period"].astype(str),
                         y=df["price_doc"]))

2. Постройте гистограмму распределения цен на квартиры с общей площадью более 50 кв.метров

In [ ]:
df = df_all[df_all["full_sq"] >= 50]

fig = go.Figure()
fig.add_trace(go.Histogram(x=df_1["price_doc"]))
fig.update_layout(bargap=0.2)

3. Визуализируйте количество проданных квартир в разных районах

In [ ]:
df = df_all.groupby("sub_area").size().reset_index(name="amount").sort_values(by="amount", ascending=False)
fig = go.Figure()
fig.add_trace(go.Bar(x=df["sub_area"],
                     y=df["amount"]))
fig.update_layout(bargap=0.2)

4. Постройте box plot отображающий распределение полных площадей квартир в Басманном районе

In [ ]:
df = df_all[df_all["sub_area"] == "Basmannoe"]

fig = go.Figure()
fig.add_trace(go.Box(y=df["full_sq"],
                     name="Full square distribution"))

5. Визуализируйте зависимость средней стоимости квартир в зависимости от этажа

In [ ]:
df = df_all.dropna(subset="floor")
df = df.groupby("floor")["price_doc"].mean().reset_index()
df["floor"] = df["floor"].astype(int)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df["floor"],
                         y=df["price_doc"]))

6. Покажите как меняется средняя площадь квартир в зависимости от района

In [ ]:
df = (df_all
      .dropna(subset="full_sq")
      .groupby("sub_area")["full_sq"]
      .mean()
      .reset_index()
      .sort_values(by="full_sq", ascending=False))

fig = go.Figure()
fig.add_trace(go.Bar(x=df["sub_area"],
                     y=df["full_sq"]))
fig.update_layout(yaxis=dict(range=[30, df["full_sq"].max() + 5]))

7. Постройте график изменения медианной цены на квартиру менее 50 кв. метров за период с 2011 до 2014 года*

\* Задание на доп. балл